# Monitoring of methane (CH4) emission point sources

## Introduction

In this notebook, we will demonstrate how to detect and monitor methane (CH₄) emissions from point sources. The approach utilizes Sentinel-2 satellite imagery to compute changes in reflectance values within the short-wave infrared (SWIR) bands. Drawing upon [recent findings from the earth observation literature](https://amt.copernicus.org/articles/14/2771/2021/), you will learn to implement a custom methane detection algorithm, enabling you to identify and track methane leakages at various sites globally.

#### Why monitor Methane (CH4)?

Methane (CH4) is a major anthropogenic greenhouse gas that is a by-product of oil & gas extraction, coal mining, large-scale animal farming, or waste disposal among other sources. The global warming potential of CH4 is [86 times that of CO2](https://amt.copernicus.org/articles/16/89/2023/) and the Intergovernmental Panel on Climate Change (IPCC) estimates that [methane is responsible for 30% of observed global warming to date](https://www.iea.org/reports/global-methane-tracker-2022/overview). Rapidly reducing leakage of CH4 into the atmosphere represents a critical component in the fight against climate change, an issue which is a key focus for policy makers around the world. At the 2021 United Nations Climate Change Conference (COP26), [The Global Methane Pledge](https://www.globalmethanepledge.org/) was initiated with the stated goal of “fast action on methane to keep a 1.5°C future within reach”. As of mid-2023, the Pledge has [150 signatories](https://www.globalmethanepledge.org/#pledges), including the US, the EU and many other nations.

#### Remote sensing of methane point sources using multispectral satellite imagery

Satellite-based methane sensing approaches typically rely on the unique transmittance characteristics of CH4. In the visible spectrum, CH4 has transmittance values equal or close to 1, meaning it is undetectable by the naked eye. Across certain wavelengths, however, methane does absorb light (transmittance <1), a property which can be exploited for detection purposes. In this notebook we will be using [Sentinel 2](https://sentinel.esa.int/web/sentinel/missions/sentinel-2) imagery, which strikes a strong balance between revisit rate (5days), spatial resolution (20m) and open access (hosted on the [AWS Registry of Open Data](https://registry.opendata.aws/sentinel-2/)).

## Setup imports

In [ ]:
! pip install rioxarray geopandas folium pystac_client --quiet

In [ ]:
import os
import boto3
import json
import rioxarray
import numpy as np
import pandas as pd
import geopandas as gpd
import math
import warnings
from shapely import wkt, geometry
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import IPython.display
import rasterio
from rasterio.plot import show
from shapely.geometry import Point, Polygon, box

session = boto3.Session()

## Define input variables

In this example, we'll monitor a known methane leak originating from an oil field in Northern Africa. This case is a standard validation scenario in remote sensing literature and is referenced, for instance, in the following [study](https://amt.copernicus.org/articles/14/2771/2021/).

The following variables define the precise location and date that we aim to monitor for emissions. Additionally, we specify a distance in meters to establish the size of a bounding box extending in each direction from the point location. Only data within this boundary will be considered for detection purposes.

In [ ]:
# coordinates and date for North Africa oil field (common reference in earth observation literature)
# see here for details: https://doi.org/10.5194/amt-14-2771-2021
point_longitude = 5.9053
point_latitude = 31.6585
date_to_monitor = "2019-11-20"

# size of bounding box in each direction around point
distance_offset_meters = 1500

## Search matching Sentinel-2 satellite imagery

The following cells create a bounding box using the specified point coordinates, and then searches for available Sentinel-2 imagery based on that bounding box and the designated monitoring date.

It utilizes the [Element84 EarthSearch STAC API](https://earth-search.aws.element84.com/v1) to retrieve satellite imagery corresponding to both the time range and the established bounding box, which delineates our area of interest.

In [ ]:
def bbox_around_point(lon, lat, distance_offset=1500, output_projection="EPSG:4326"):
    """
    Create a square bounding box of size distance_offset*2 by distance_offset*2
    with the provided coordinates as centroid
    """

    gs = gpd.GeoSeries(wkt.loads(f"POINT ({lon} {lat})"))
    gdf = gpd.GeoDataFrame(geometry=gs)
    gdf.crs = "EPSG:4326"
    gdf = gdf.to_crs("EPSG:3857")  # <-- change crs to a projection crs, e.g., Web Mercator
    res = gdf.buffer(
        distance=distance_offset,
        cap_style=3,  # <-- ensure sharp edges
    )

    if output_projection != "EPSG:3857":
        res = res.to_crs(output_projection)
    
    return res.iloc[0]

In [ ]:
# generate bounding box and extract polygon coordinates
aoi_geometry = bbox_around_point(point_longitude, point_latitude, distance_offset_meters)
geometry.mapping(aoi_geometry)

In [ ]:
# Create a Shapely Polygon object
polygon = Polygon(geometry.mapping(aoi_geometry)['coordinates'][0])

In [ ]:
def as_iso_date(date_str):
    # convert to datetime to validate format
    """
    Convert time string to datetime
    """
    return datetime.strptime(date_str, "%Y-%m-%d").strftime("%Y-%m-%d")


def get_sentinel2_meta_data(target_date, polygon):
    """
    Search Sentinel 2 data collection for target_date
    and collect results including meta data in a dictionary
    """

    # now uses STAC
    catalog = Client.open("https://earth-search.aws.element84.com/v1")
    
    query = catalog.search(
        collections=["sentinel-2-l2a"], datetime=target_date, limit=100, bbox=polygon.bounds
        )
    
    items = list(query.items())
    
    # Convert STAC items into a GeoJSON FeatureCollection
    stac_json = query.item_collection_as_dict()
    items = list(query.item_collection())

    return items

In [ ]:
import folium
import folium.plugins
import geopandas as gpd
import shapely.geometry
import yaml
from branca.element import Figure
from IPython.display import HTML, display
from pystac_client import Client

In [ ]:
catalog = Client.open("https://earth-search.aws.element84.com/v1")

query = catalog.search(
    collections=["sentinel-2-l2a"], datetime="2019-11-20", limit=100, bbox=polygon.bounds
)

items = list(query.items())
print(f"Found: {len(items):d} datasets")

# Convert STAC items into a GeoJSON FeatureCollection
stac_json = query.item_collection_as_dict()

In [ ]:
items = list(query.item_collection())

print(f"Number of items: {len(items)}")
for item in items:
    print(f"- {item.id}")

In [ ]:
collection = catalog.get_child("sentinel-2-l2a")

a_item = collection.get_item("S2A_31RGQ_20191120_1_L2A", recursive=False)

thumbnail_asset = a_item.assets['thumbnail']
thumbnail_href = thumbnail_asset.href

thumbnail_href

### Inspect a single Sentinel-2 tile

We will now inspect one of the items returned from our query. Each item offers a list of assets for all the Sentinel-2 bands, along with additional metadata. For instance, we can view the thumbnail, which gives an overview of the area covered by this specific Sentinel-2 tile.

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=rasterio.errors.NotGeoreferencedWarning)
    src_thumbnail = rasterio.open(thumbnail_href)

fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(1, 1, 1)
show(src_thumbnail, ax=ax)
plt.show()

We can also visualize a True Color Image (TCI) that is included in the dataset.

The Sentinel-2 mission defines a TCI as follows (more details here):

The TCI is an RGB image created using the B02 (Blue), B03 (Green), and B04 (Red) bands. Reflectances are encoded in a range between 1 and 255, with 0 being reserved for 'No Data'. A saturation level of 255 digital counts corresponds to a level of 3558 for L1C products and 2000 for L2A products (reflectance values of 0.3558 and 0.2, respectively).

Mapping to fixed saturation values can result in over-saturated images in locations with substantial sun exposure, a situation evident in this desert region.

Nevertheless, we can utilize the TCI to obtain a general view of the data coverage and to zoom in closer to the area where the point source is situated.

In [ ]:
cog_asset = a_item.assets['visual']
cog_href = cog_asset.href

cog_href

In [ ]:
src_visual = rasterio.open(cog_href)
data_visual = src_visual.read()


fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1)
# show(data_visual[:, 8000:10000, 500:2500], ax=ax)
show(data_visual, ax=ax)
plt.show()

### Visualize the area of interest

Since the TCI is slightly oversaturated, we will now use the individual red, green, and blue bands to create a true color image by combining these bands. In this process, we will normalize the reflectance values to prevent oversaturation.

Next, we will zoom in directly to the actual point source location to display only the surrounding area.

In [ ]:
def s2_tile_id_to_cog_path(tile_id):
    """
    Generate s3 URL for S2 L2A product (i.e., the cloud-optimized GeoTiff) from tile ID
    """
    parts = tile_id.split("_")
    s2_qualifier = "{}/{}/{}/{}/{}/{}".format(
        parts[1][0:2],
        parts[1][2],
        parts[1][3:5],
        parts[2][0:4],
        str(int(parts[2][4:6])),
        "_".join(parts),
    )

    return f"https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/{s2_qualifier}"


def get_rgb_bands(lon, lat, s2_tile_id, distance_offset=1500, output_projection="EPSG:4326"):
    """
    Extract red, green, blue (RGB) bands from GeoTiff,
    reproject to desired CRS and collect in array
    """
    aoi_geometry = bbox_around_point(lon, lat, distance_offset, output_projection)

    s2_cog_prefix = s2_tile_id_to_cog_path(s2_tile_id)
    blue = rioxarray.open_rasterio(f"{s2_cog_prefix}/B02.tif", masked=True)
    green = rioxarray.open_rasterio(f"{s2_cog_prefix}/B03.tif", masked=True)
    red = rioxarray.open_rasterio(f"{s2_cog_prefix}/B04.tif", masked=True)

    bands = []
    for band in [red, green, blue]:
        kwargs = {"nodata": np.nan}
        band_reprojected = band.rio.reproject(output_projection, **kwargs)
        bands.append(band_reprojected.rio.clip(geometries=[geometry.mapping(aoi_geometry)]))
    return bands


def normalize_rgb_bands(rgb_bands):
    """
    Re-scale/normalize rgb bands to range [0,255]
    """
    bands = []
    for band in rgb_bands:
        img_arr = band.to_numpy()
        normalized = img_arr / np.amax(img_arr)
        normalized = normalized * 255
        bands.append(normalized.astype(np.uint8)[0])
    return np.array(bands)

In [ ]:
sentinel2_tile_id = items[0].id

In [ ]:
preview_distance_offset = 3000

visual_bands = get_rgb_bands(
    point_longitude, point_latitude, sentinel2_tile_id, distance_offset=preview_distance_offset
)
rgb = normalize_rgb_bands(visual_bands).transpose(1, 2, 0)

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1)
plt.imshow(rgb)

height = rgb.shape[0]
width = rgb.shape[1]
y_m_per_pixel = preview_distance_offset * 2 / height
x_m_per_pixel = preview_distance_offset * 2 / width

y_distance_offset = distance_offset_meters / y_m_per_pixel
x_distance_offset = distance_offset_meters / x_m_per_pixel

p = patches.Circle(
    (width / 2, height / 2), radius=3, linewidth=2, edgecolor="black", facecolor="none"
)

rect = patches.Rectangle(
    (width / 2 - x_distance_offset, height / 2 - y_distance_offset),
    x_distance_offset * 2,
    y_distance_offset * 2,
    linewidth=1,
    edgecolor="y",
    facecolor="none",
)
ax.add_patch(p)
ax.add_patch(rect)

ax.annotate(
    "Monitored $CH_4$\n Point Source",
    xy=(width / 2 + 5, height / 2 - 5),
    xytext=(width / 2 + 25, height / 2 - 25),
    arrowprops=dict(facecolor="black", shrink=0.05),
    ha="left",
)

plt.show()

## Approximate representative baseline date and corresponding Sentinel-2 tile

Our detection approach relies on observing fractional changes in the [top-of-the-atmosphere (TOA)](https://www.un-spider.org/node/10958) [short-wave infrared (SWIR)](https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-2-msi/resolutions/spatial) reflectance. To facilitate this, it is critical to identify a reliable baseline where the methane presence is minimal or non-existent. Establishing such a baseline can rapidly devolve into a tedious search process filled with trial and error. Fortunately, employing adept heuristics can significantly automate this process. 

One heuristic that has worked well in the investigated cases operates as follows: for the predetermined number of past days specified as `day_offset=n`, retrieve all available satellite imagery, eliminate any cloud cover, and clip the imagery to match the area of interest (AOI) in focus. Subsequently, calculate the average band-12 reflectance across the AOI and identify the Sentinel tile ID of the image showcasing the highest average reflectance in the band-12. The following functions are used to implement this approach.

In [ ]:
def download_from_s3(s3_obj_url, local_dir, local_file_fn=None, requester_pays=False):
    """
    Download Sentinel 2 L1C product (as identified by s3_obj_url)
    to local file share
    """
    os.makedirs(local_dir, exist_ok=True)
    if local_file_fn is None:
        local_file_path = os.path.join(local_dir, s3_obj_url.split("/")[-1])
    else:
        local_file_path = local_file_fn(local_dir, s3_obj_url)

    target_bucket_name = s3_obj_url.split("/")[2]
    target_bucket_ob_key = "/".join(s3_obj_url.split("/")[3:])

    s3_bucket = session.resource("s3").Bucket(target_bucket_name)
    if requester_pays:
        s3_bucket.download_file(
            target_bucket_ob_key, local_file_path, ExtraArgs={"RequestPayer": "requester"}
        )
    else:
        s3_bucket.download_file(target_bucket_ob_key, local_file_path)
    return local_file_path


def s2_tile_id_to_s2l1c_s3_url(tile_id):
    """
    Generate s3 URL for S2 L1C product from tile ID
    """
    parts = tile_id.split("_")
    s2l1c_qualifier = "{}/{}/{}/{}/{}/{}/0".format(
        parts[1][0:2],
        parts[1][2],
        parts[1][3:5],
        parts[2][0:4],
        str(int(parts[2][4:6])),
        str(int(parts[2][6:8])),
    )
    return f"s3://sentinel-s2-l1c/tiles/{s2l1c_qualifier}/"


def get_s2l1c_band_data_xarray(s2_tile_id, band_id, clip_geometry=None, cloud_mask=True):
    """
    Download S2 L1C data for given tile ID, optionally apply cloud mask and clip to AOI (clip geometry),
    reproject to target WGS84 (EPSG:4326) coordinate reference system (CRSs)
    """

    data_dir = "data/s2l1c/"
    s3_prefix = s2_tile_id_to_s2l1c_s3_url(s2_tile_id)

    local_tile_path = download_from_s3(
        f"{s3_prefix}{band_id}.jp2",
        data_dir,
        local_file_fn=lambda local_dir, s3_obj_url: os.path.join(
            data_dir, f"{s2_tile_id}_{band_id}.jp2"
        ),
        requester_pays=True,
    )

    band_data_epsg_32631 = rioxarray.open_rasterio(local_tile_path, masked=True)

    if cloud_mask:
        s2_cog_prefix = s2_tile_id_to_cog_path(s2_tile_id)
        scl_mask = rioxarray.open_rasterio(f"{s2_cog_prefix}/SCL.tif", masked=True)

        # no interpolation needed, as long as target band and scl mask have identical spatial resolution
        band_data_epsg_32631 = band_data_epsg_32631.where(
            (scl_mask != 8) & (scl_mask != 9) & (scl_mask != 10)
        )

    kwargs = {"nodata": np.nan}
    band_data_epsg_4326 = band_data_epsg_32631.rio.reproject("EPSG:4326", **kwargs)
    if clip_geometry is None:
        return band_data_epsg_4326
    return band_data_epsg_4326.rio.clip(geometries=[geometry.mapping(clip_geometry)])

In [ ]:
def approximate_best_baseline_date(
    lon, lat, date_to_monitor, distance_offset=1500, cloud_mask=True, day_offset=30
):
    """
    For the past n=day_offset days, do: retrive Sentinel 2 image,
    remove clouds, compute mean reflectance of SWIR band (b-12) across scene,
    return Sentinel 2 tile id (and acquisition data) of image
    with highest mean SWIR values
    """
    # initialize AOI and other parameters
    aoi_geometry = bbox_around_point(lon, lat, distance_offset)

    BAND_12_SWIR22 = "B12"

    max_mean_swir = None
    ref_s2_tile_id = None
    ref_target_date = date_to_monitor

    # loop over n=day_offset previous days
    for day_delta in range(-1 * day_offset, 0):
        date_time_obj = datetime.strptime(date_to_monitor, "%Y-%m-%d")
        target_date = (date_time_obj + timedelta(days=day_delta)).strftime("%Y-%m-%d")

        # get Sentinel-2 tiles for current date
        s2_tiles_for_target_date = get_sentinel2_meta_data(target_date, aoi_geometry)

        # loop over available tiles for current date
        for s2_tile_meta in s2_tiles_for_target_date:
            # s2_tile_id_to_test = s2_tile_meta["Id"]
            s2_tile_id_to_test = s2_tile_meta.id
            target_band_data = get_s2l1c_band_data_xarray(
                s2_tile_id_to_test,
                BAND_12_SWIR22,
                clip_geometry=aoi_geometry,
                cloud_mask=cloud_mask,
            )

            # compute mean reflectance of SWIR band
            mean_swir = target_band_data.sum() / target_band_data.count()
            visible_area_ratio = target_band_data.count() / (
                target_band_data.shape[1] * target_band_data.shape[2]
            )

            # ensure the visible/non-clouded area is adequately large
            if visible_area_ratio <= 0.7:
                continue

            # update maximum ref_s2_tile_id and ref_target_date if applicable
            if max_mean_swir is None or mean_swir > max_mean_swir:
                max_mean_swir = mean_swir
                ref_s2_tile_id = s2_tile_id_to_test
                ref_target_date = target_date

    return (ref_s2_tile_id, ref_target_date)

We can now use the `approximate_best_baseline_date` function to identify a suitable baseline candidate. We'll utilize the identified baseline for the computation of the change in fractional reflectances in the next steps.

In [ ]:
baseline_s2_tile_id, baseline_date = approximate_best_baseline_date(
    point_longitude,
    point_latitude,
    date_to_monitor,
    day_offset=45,
    distance_offset=distance_offset_meters,
)

print(baseline_date)
print(baseline_s2_tile_id)

## Compute change in fractional reflectance

The detection method we are implementing involves calculating the fractional change in top-of-the-atmosphere (TOA) reflectance, denoted as $\Pi \rho$. This measure encompasses the reflectance recorded by the Sentinel-2 mission across two distinct satellite passes and two SWIR bands. These passes include a baseline, where no methane is presumed present (referred to as `base`), and a monitoring pass where there is suspicion of an active methane point source (referred to as `monitor`). Mathematically, we can express this as follows:

$$ \Delta \rho = {c^{monitor} \rho^{monitor}_{b12} - \rho^{monitor}_{b11} \over \rho^{monitor}_{b11}}-{c^{base} \rho^{base}_{b12} - \rho^{base}_{b11} \over \rho^{base}_{b11}}$$

where $\rho$ represents the TOA reflectance as measured by Sentinel-2. The correction factors $c^{monitor}$ and $c^{base}$ are determined by regressing TOA reflectance values of band-12 against those of band-11 across the entire scene, denoted mathematically as $\rho_{b11} = c × \rho_{b12}$

The following functions implement the described approach.

In [ ]:
BAND_11_SWIR16 = "B11"
BAND_12_SWIR22 = "B12"


def compute_correction_factor(tif_y, tif_x):
    """
    Compute a correction factor c via linera regression
    with the following equation: tif_y = c*tif_x
    """

    # get flattened arrays for regression
    y = np.array(tif_y.values.flatten())
    x = np.array(tif_x.values.flatten())

    np.nan_to_num(y, copy=False)
    np.nan_to_num(x, copy=False)

    assert y.shape == x.shape, "Shapes of two provided TIFs are incompatible!"

    # fit linear model using least squares regression
    x = x[:, np.newaxis]
    c, _, _, _ = np.linalg.lstsq(x, y, rcond=None)

    return c[0]


def compute_corrected_fractional_reflectance_change(
    l1_b11_base, l1_b12_base, l1_b11_monitor, l1_b12_monitor, correction=True
):
    """
    Compute multi-band-multi-pass fractional reflectance change
    between base and monitoring scene
    """
    # get correction factor
    c_monitor = compute_correction_factor(tif_y=l1_b11_monitor, tif_x=l1_b12_monitor)
    c_base = compute_correction_factor(tif_y=l1_b11_base, tif_x=l1_b12_base)

    # get corrected fractional reflectance change
    if correction:
        frac_change = ((c_monitor * l1_b12_monitor - l1_b11_monitor) / l1_b11_monitor) - (
            (c_base * l1_b12_base - l1_b11_base) / l1_b11_base
        )
    else:
        frac_change = ((l1_b12_monitor - l1_b11_monitor) / l1_b11_monitor) - (
            (l1_b12_base - l1_b11_base) / l1_b11_base
        )
    return frac_change


def run_full_fractional_reflectance_change_routine(
    lon, lat, date_monitor, baseline_s2_tile_id, distance_offset=1500, cloud_mask=True
):
    """
    End-to-end routine for computing corrected fractional reflectance change (SWIR spectrum)
    given a date to monitor and a baseline Sentinel 2 tile ID
    """

    s2_tile_id_base = baseline_s2_tile_id
    aoi_geometry = bbox_around_point(lon, lat, distance_offset)

    s2_meta_monitor = get_sentinel2_meta_data(date_monitor, aoi_geometry)
    grid_id = s2_tile_id_base.split("_")[1]

    s2_tile_id_monitor = list(filter(lambda x: f"_{grid_id}_" in x.id, s2_meta_monitor))[0].id
    # print(s2_tile_id_monitor)

    l1_swir16_b11_base = get_s2l1c_band_data_xarray(
        s2_tile_id_base, BAND_11_SWIR16, clip_geometry=aoi_geometry, cloud_mask=cloud_mask
    )
    l1_swir22_b12_base = get_s2l1c_band_data_xarray(
        s2_tile_id_base, BAND_12_SWIR22, clip_geometry=aoi_geometry, cloud_mask=cloud_mask
    )
    l1_swir16_b11_monitor = get_s2l1c_band_data_xarray(
        s2_tile_id_monitor, BAND_11_SWIR16, clip_geometry=aoi_geometry, cloud_mask=cloud_mask
    )
    l1_swir22_b12_monitor = get_s2l1c_band_data_xarray(
        s2_tile_id_monitor, BAND_12_SWIR22, clip_geometry=aoi_geometry, cloud_mask=cloud_mask
    )

    mb_mp = compute_corrected_fractional_reflectance_change(
        l1_swir16_b11_base, l1_swir22_b12_base, l1_swir16_b11_monitor, l1_swir22_b12_monitor
    )
    return mb_mp.rio.set_attrs(
        {
            "s2_tile_id_baseline": s2_tile_id_base,
            "s2_tile_id_monitor": s2_tile_id_monitor,
        }
    )

Executing the `run_full_fractional_reflectance_change_routine` with the parameters we determined earlier generates the fractional change in SWIR TOA reflectance, represented as an xarray.DataArray. To initially inspect the results visually, we can execute a simple plot() function on this DataArray. Our method reveals a sizable methane plume at the center of the AOI, a detail that was not visible in the RGB plot.

In [ ]:
change_reflectance = run_full_fractional_reflectance_change_routine(
    point_longitude,
    point_latitude,
    date_to_monitor,
    baseline_s2_tile_id,
    distance_offset=distance_offset_meters,
)

In [ ]:
plot = change_reflectance.plot(cmap="RdBu")

## Compare visual imagery with detected plume mask

As a final step, we will extract the identified methane plume and overlay it on a raw RGB satellite image to contextualize it geographically. This is achievable through thresholding.

In our instance, applying a threshold of -0.025 to the fractional change in reflectance yields good results for the observed scene. Nevertheless, it is important to note that the optimal threshold may vary from scene to scene, requiring calibration tailored to your specific use case. The following cell creates a visualization that incorporates the original RGB image, the computed plume mask, and a composite image that shows the methane plume within its geographical context.

In [ ]:
output_projection = "EPSG:3857"  # Web Mercator projection (EPSG:3857) for visualization
treshold_value = -0.025

visual_bands = get_rgb_bands(
    point_longitude,
    point_latitude,
    change_reflectance.attrs["s2_tile_id_monitor"],
    distance_offset=distance_offset_meters,
    output_projection=output_projection,
)

rgb = normalize_rgb_bands(visual_bands).transpose(1, 2, 0)

change_reflectance_reprojected = change_reflectance.rio.reproject(output_projection)

with warnings.catch_warnings():
    warnings.simplefilter(action="ignore", category=FutureWarning)

    cr_interp = change_reflectance_reprojected.interp(
        x=visual_bands[0]["x"], y=visual_bands[0]["y"]
    )
    cr = cr_interp.to_numpy()[0]

cr_masked = cr.copy()
cr_masked[cr_masked > treshold_value] = np.nan

m = np.ma.array(cr_masked, mask=cr_masked == np.nan)

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16, 10), sharex=True, sharey=True)

ax = axes[0]
ax.imshow(rgb)
ax.set_title("Raw Satellite Image\n (Visible Spectrum)", fontsize=14)

ax = axes[1]
ax.imshow(cr, cmap="RdBu")  # <-- uses interpolated data
ax.set_title("Fractional Change in TOA Reflectance\n (SWIR Spectrum)", fontsize=14)

ax = axes[2]
ax.imshow(rgb)
ax.imshow(m, alpha=0.8, cmap="RdBu")
ax.set_title("Relative $CH_4$ Plume Intensity overlaid\n on Raw Satellite Image", fontsize=14)

for ax in axes:
    ax.grid(True)
    ax.set_xticklabels([])
    ax.set_yticklabels([])

txt = f"Location: (lon: {point_longitude}, lat: {point_latitude}); Reference Tile Id: {change_reflectance.attrs['s2_tile_id_baseline']}; Monitoring Tile Id: {change_reflectance.attrs['s2_tile_id_monitor']}; Scale: {distance_offset_meters*2}x{distance_offset_meters*2}m"
# plt.figtext(0.015, 0.25, txt, wrap=False, horizontalalignment="left", fontsize=11)
plt.figtext(0.015, 0.225, txt, wrap=False, horizontalalignment="left", fontsize=11)

plt.tight_layout()

plt.show()